<a href="https://colab.research.google.com/github/axdecimal/CNNmodelling/blob/master/Toxo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
zip_adres = "/content/drive/MyDrive/Colab Notebook/Data2/toxo.zip"
!cp "{zip_adres}" .

In [28]:
import random
import numpy as np
import tensorflow as tf
import copy
import math

m = 2


def initialize_U(num_data, num_cluster):
    U = []
    for data in range(num_data):
        u_data = []
        sum_random = 0.0
        for cluster in range(num_cluster):
            u_data.append(random.random())
            sum_random += u_data[-1]
        for cluster in range(num_cluster):
            u_data[cluster] /= sum_random
        U.append(u_data)

    return U


def initialize_C(data, U):
    """
    :param data: features of the data
    :param U:
    :param m:
    :return:
    """
    num_cluster = len(U[0])
    C = []
    for j in range(num_cluster):
        current_cluster_center = []
        for i in range(len(data[0])):
            dummy_sum_num = 0.0
            dummy_sum_dum = 0.0
            for k in range(len(data)):
                # 分子
                dummy_sum_num += (U[k][j] ** m) * data[k][i]
                # 分母
                dummy_sum_dum += (U[k][j] ** m)
            # 第i列的聚类中心
            current_cluster_center.append(dummy_sum_num / dummy_sum_dum)
            # 第j簇的所有聚类中心
        C.append(current_cluster_center)
    return C


def calculate_J(data, U, C):
    """
    计算目标函数J
    :param data:
    :param C:
    :return:
    """
    J = 0.0
    for num_C in range(len(C)):
        for num_data in range(len(data)):
            J += (np.power(U[num_data][num_C], m) * np.square(np.subtract(C[num_C], data[num_data])))
    return J


def distance(point, center):
    """
    该函数计算2点之间的距离（作为列表）。我们指欧几里德距离。        闵可夫斯基距离
    """
    if len(point) != len(center):
        return -1
    dummy = 0.0
    for i in range(0, len(point)):
        dummy += abs(point[i] - center[i]) ** 2
    return math.sqrt(dummy)


def end_conditon(U, U_old):
    """
    结束条件。当U矩阵随着连续迭代停止变化时，触发结束
    """
    epsm = 0.00000001
    for i in range(0, len(U)):
        for j in range(0, len(U[0])):
            if abs(U[i][j] - U_old[i][j]) > epsm:
                return False
    return True

def cal_U(data, C, cluster_number, m):
    U=[0.0,0.0]
    distance_matrix = [0.0,0.0]
    #current = []
    for j in range(0, cluster_number):
       #print('cluster_j:',j)
       distance_matrix[j]=distance(data,C[j])
       #current.append(distance(data, C[j]))
       #distance_matrix.append(current)
    # 更新U
    for j in range(0, cluster_number):
        dummy = 0.0
        for k in range(0, cluster_number):
            dummy += (distance_matrix[j] / distance_matrix[k]) ** (2 / (m - 1))
        U[j] = 1 / dummy
    return U
def updata_U(checkpoints_dir, data, U, UC_name):
    cluster_number = len(U[0])

    for iter in range(100):
        print('第%d 次迭代更新U' % iter)
        U_old = copy.deepcopy(U)
        C = []
        for j in range(0, cluster_number):
            current_cluster_center = []
            for i in range(0, len(data[0])):
                dummy_sum_num = 0.0
                dummy_sum_dum = 0.0
                for k in range(0, len(data)):
                    dummy_sum_num += (U[k][j] ** m) * data[k][i]
                    dummy_sum_dum += (U[k][j] ** m)
                current_cluster_center.append(dummy_sum_num / dummy_sum_dum)
            C.append(current_cluster_center)

        distance_matrix = []
        for i in range(0, len(data)):
            current = []
            for j in range(0, cluster_number):
                current.append(distance(data[i], C[j]))
            distance_matrix.append(current)
        # 更新U
        for j in range(0, cluster_number):
            for i in range(0, len(data)):
                dummy = 0.0
                for k in range(0, cluster_number):
                    dummy += (distance_matrix[i][j] / distance_matrix[i][k]) ** (2 / (m - 1))
                U[i][j] = 1 / dummy

        if end_conditon(U, U_old):
             print("结束聚类")
             break

    np.savetxt(checkpoints_dir + "/Uy" + UC_name + '.txt', U, fmt="%.20f", delimiter=",")
    np.savetxt(checkpoints_dir + "/Cy" + UC_name + '.txt', C, fmt="%.20f", delimiter=",")

    return U, C


def getSubU(U, idx):
    target = []
    for i in idx:
        target.append(U[i])

    return target

def initialize_UC_test(x_len,x_data,y_len,y_data, UC_name,checkpoints_dir):
    #x_images, x_id_list, x_len, x_labels = dr.get_source_batch(0, 256, 256, source_dir=source_dir)
    #y_images, y_id_list, y_len, y_labels = dr.get_target_batch(0, 256, 256, target_dir=target_dir)
    #print('x_len',len(x_images))
    #print('y_len',len(y_images))
    #x = tf.placeholder(tf.float32, [None, 256, 256, 3])
    #y = tf.placeholder(tf.float32, [None, 256, 256, 3])

    # C_initial = Classifier('C', True, reuse=True)
    #fx = C_initial(x)
    #fy = C_initial(y)

    #with tf.Session() as sess:
    #    tf.global_variables_initializer().run()

    #    x_data = []
    #    y_data = []

    #    for img in x_images:
    #        data = sess.run(fx, feed_dict={x: [img]})
    #        x_data.append(data[0])
    #    for img in y_images:
    #        data = sess.run(fy, feed_dict={y: [img]})
    #        y_data.append(data[0])
    #print('y_data:',np.sum(y_data,axis=1))
    Ux = initialize_U(x_len, 1)
    Uy = initialize_U(y_len, 2)
    Cx = initialize_C(x_data, Ux)
    Cy = initialize_C(y_data, Uy)
    print('Cx:',np.shape(Cx))
    print('Cy:',np.shape(Cy))

    return Ux, Uy, Cx, Cy
def initialize_UC(checkpoints_dir, C_initial, UC_name, source_dir, target_dir):
    x_images, x_id_list, x_len, x_labels = dr.get_source_batch(0, 256, 256, source_dir=source_dir)
    y_images, y_id_list, y_len, y_labels = dr.get_target_batch(0, 256, 256, target_dir=target_dir)
    #print('x_len',len(x_images))
    #print('y_len',len(y_images))
    x = tf.placeholder(tf.float32, [None, 256, 256, 3])
    y = tf.placeholder(tf.float32, [None, 256, 256, 3])

    # C_initial = Classifier('C', True, reuse=True)
    fx = C_initial(x)
    fy = C_initial(y)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        x_data = []
        y_data = []

        for img in x_images:
            data = sess.run(fx, feed_dict={x: [img]})
            x_data.append(data[0])
        for img in y_images:
            data = sess.run(fy, feed_dict={y: [img]})
            y_data.append(data[0])
    print('y_data:',np.sum(y_data,axis=1))
    Ux = initialize_U(x_len, 1)
    Uy = initialize_U(y_len, 2)
    Cx = initialize_C(x_data, Ux)
    Cy = initialize_C(y_data, Uy)

    np.savetxt(checkpoints_dir + "/Ux" + UC_name + '.txt', Ux, fmt="%.20f", delimiter=",")
    np.savetxt(checkpoints_dir + "/Uy" + UC_name + '.txt', Uy, fmt="%.20f", delimiter=",")
    np.savetxt(checkpoints_dir + "/Cx" + UC_name + '.txt', Cx, fmt="%.20f", delimiter=",")
    np.savetxt(checkpoints_dir + "/Cy" + UC_name + '.txt', Cy, fmt="%.20f", delimiter=",")

    return Ux, Uy, Cx, Cy, x_data, y_data, y_labels


def initialize_UC2(C_initial, y_images):
    # y_images, y_id_list, y_len, y_labels = dr.get_target_batch(0, 256, 256, ".png", target_dir=target_dir)

    y = tf.placeholder(tf.float32, [None, 256, 256, 3])

    fy = C_initial(y)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        y_data = []

        for img in y_images:
            data = sess.run(fy, feed_dict={y: [img]})
            y_data.append(data[0])

    return y_data


def get_nearst_accuracy(datas, C, label):
    distances = []
    for data in datas:
        distance = []
        distance_1 = np.mean(np.sqrt(np.square(np.subtract(data, C[0]))))
        distance_2 = np.mean(np.sqrt(np.square(np.subtract(data, C[1]))))
        distance.append(distance_1)
        distance.append(distance_2)
        distances.append(distance)

    accuracy = 0
    wrong = 0
    wrong_cell = 0
    num_cell = 0
    wrong_toxo = 0
    num_toxo = 0
    for index in range(len(distances)):
        arg = np.argmin(distances[index])
        if label[index] == 0:
            num_toxo += 1
        else:
            num_cell += 1
        if arg == label[index]:
            accuracy += 1
        else:
            wrong += 1
            if label[index] == 0:
                wrong_toxo += 1
            else:
                wrong_cell += 1

    return accuracy / len(datas), wrong_toxo / num_toxo, wrong_cell / num_cell, num_toxo, num_cell
if __name__ == '__main__':
    # initialize_UC()
    pass

In [ ]:
import data_reader as dr
import fuzzy
import numpy as np
import random
import copy
import math
from compute_accuracy import computeAccuracy

m = 2
def initialize_U(num_data, num_cluster):
    U = []
    for data in range(num_data):
        u_data = []
        sum_random = 0.0
        for cluster in range(num_cluster):
            u_data.append(random.random())
            sum_random += u_data[-1]
        for cluster in range(num_cluster):
            u_data[cluster] /= sum_random
        U.append(u_data)

    return U


def initialize_C(data, U):
    """
    :param data: features of the data
    :param U:
    :param m:
    :return:
    """
    num_cluster = len(U[0])
    C = []
    for j in range(num_cluster):
        current_cluster_center = []
        for i in range(len(data[0])):
            dummy_sum_num = 0.0
            dummy_sum_dum = 0.0
            for k in range(len(data)):
                # 分子
                dummy_sum_num += (U[k][j] ** m) * data[k][i]
                # 分母
                dummy_sum_dum += (U[k][j] ** m)
            # 第i列的聚类中心
            current_cluster_center.append(dummy_sum_num / dummy_sum_dum)
            # 第j簇的所有聚类中心
        C.append(current_cluster_center)
    return C
def initialize_UC(y_data):
    print('y_data:',np.sum(y_data,axis=1))
    y_len=len(y_data)
    #Ux = initialize_U(x_len, 1)
    Uy = initialize_U(y_len, 2)
    #Cx = initialize_C(x_data, Ux)
    Cy = initialize_C(y_data, Uy)
    #print('Cx:',np.shape(Cx))
    print('Cy:',np.shape(Cy))
    return Uy,Cy
def end_conditon(U, U_old):
    """
    结束条件。当U矩阵随着连续迭代停止变化时，触发结束
    """
    epsm = 0.000000001
    for i in range(0, len(U)):
        for j in range(0, len(U[0])):
            if abs(U[i][j] - U_old[i][j]) > epsm:
                return False
    return True
def calculate_J(data, U, C):
    """
    计算目标函数J
    :param data:
    :param C:
    :return:
    """
    J = 0.0
    for num_C in range(len(C)):
        for num_data in range(len(data)):
            J += (np.power(U[num_data][num_C], m) * np.square(np.subtract(C[num_C], data[num_data])))
    return J


def distance(point, center):
    """
    该函数计算2点之间的距离（作为列表）。我们指欧几里德距离。        闵可夫斯基距离
    """
    if len(point) != len(center):
        return -1
    dummy = 0.0
    for i in range(0, len(point)):
        dummy += abs(point[i] - center[i]) ** 2
    return math.sqrt(dummy)

def updata_U(saveDir, data, U):
    cluster_number = len(U[0])

    for iter in range(100):
        print('第%d 次迭代更新U' % iter)
        U_old = copy.deepcopy(U)
        C = []
        for j in range(0, cluster_number):
            current_cluster_center = []
            for i in range(0, len(data[0])):
                dummy_sum_num = 0.0
                dummy_sum_dum = 0.0
                for k in range(0, len(data)):
                    dummy_sum_num += (U[k][j] ** m) * data[k][i]
                    dummy_sum_dum += (U[k][j] ** m)
                current_cluster_center.append(dummy_sum_num / dummy_sum_dum)
            C.append(current_cluster_center)

        distance_matrix = []
        for i in range(0, len(data)):
            current = []
            for j in range(0, cluster_number):
                current.append(distance(data[i], C[j]))
            distance_matrix.append(current)
        # 更新U
        for j in range(0, cluster_number):
            for i in range(0, len(data)):
                dummy = 0.0
                for k in range(0, cluster_number):
                    dummy += (distance_matrix[i][j] / distance_matrix[i][k]) ** (2 / (m - 1))
                U[i][j] = 1 / dummy

        if end_conditon(U, U_old):
             print("结束聚类")
             break

    np.savetxt(saveDir + "/U.txt", U, fmt="%.20f", delimiter=",")
    np.savetxt(saveDir + "/C.txt", C, fmt="%.20f", delimiter=",")
    return U, C
if __name__=='__main__':
    #x_path="/home/root123/data/datasets/source/banana/"
    model='20190416-2037'
    dir='./checkpoints/+'+model+'/max/'
    y_path="/home/root123/data/datasets/target/toxo40/"
    #x_images, x_id_list, x_len, x_labels = dr.get_source_batch(0, 256, 256, source_dir=x_path)
    y_images, y_id_list, y_len, y_labels = dr.get_target_batch(0, 256, 256, target_dir=y_path)

    feature=np.load(dir+'feature_fcgan.npy')
    Uy,Cy= initialize_UC(feature)
    U,C=updata_U(dir,feature,Uy)
    print(np.shape(U))
    print(np.shape(C))
    
    y, y_idx_list, y_data_n, y_labels = dr.get_target_batch(0, 224, 224, target_dir=y_path)
    #Uy = np.loadtxt('checkpoints/' + model + '/Uy' + UC_name + '.txt', delimiter=",")
    #Uy = np.loadtxt(dir+'/U.txt', delimiter=",")
    accuracy=computeAccuracy(U,y_labels,0.5)
    print('accuracy:',accuracy)

In [32]:
import tensorflow as tf



class Generator:
    def __init__(self, name, is_training, ngf=64, norm='instance', image_size=128):
        self.name = name
        self.reuse = False
        self.ngf = ngf
        self.norm = norm
        self.is_training = is_training
        self.image_size = image_size

    def __call__(self, input):
        """
        Args:
          input: batch_size x width x height x 3
        Returns:
          output: same size as input
        """
        with tf.variable_scope(self.name):
            # conv layers
            c7s1_32 = ops.c7s1_k(input, self.ngf, is_training=self.is_training, norm=self.norm,
                                 reuse=self.reuse, name='c7s1_32')  # (?, w, h, 32)
            d64 = ops.dk(c7s1_32, 2 * self.ngf, is_training=self.is_training, norm=self.norm,
                         reuse=self.reuse, name='d64')  # (?, w/2, h/2, 64)
            d128 = ops.dk(d64, 4 * self.ngf, is_training=self.is_training, norm=self.norm,
                          reuse=self.reuse, name='d128')  # (?, w/4, h/4, 128)

            if self.image_size <= 128:
                # use 6 residual blocks for 128x128 images
                res_output = ops.n_res_blocks(d128, reuse=self.reuse, n=6)  # (?, w/4, h/4, 128)
            else:
                # 9 blocks for higher resolution
                res_output = ops.n_res_blocks(d128, reuse=self.reuse, n=9)  # (?, w/4, h/4, 128)

            # fractional-strided convolution
            u64 = ops.uk(res_output, 2 * self.ngf, is_training=self.is_training, norm=self.norm,
                         reuse=self.reuse, name='u64')  # (?, w/2, h/2, 64)
            u32 = ops.uk(u64, self.ngf, is_training=self.is_training, norm=self.norm,
                         reuse=self.reuse, name='u32', output_size=self.image_size)  # (?, w, h, 32)

            # conv layer
            # Note: the paper said that ReLU and _norm were used
            # but actually tanh was used and no _norm here
            output = ops.c7s1_k(u32, 3, norm=None,
                                activation='tanh', reuse=self.reuse, name='output')  # (?, w, h, 3)
        # set reuse=True for next call
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)

        return output

    def sample(self, input):
        image = utils.batch_convert2int(self.__call__(input))
        image = tf.image.encode_jpeg(tf.squeeze(image, [0]))
        return image

In [38]:
import tensorflow as tf
import numpy as np



class CycleGAN:
    def __init__(self,
                 batch_size=1,
                 image_size=256,
                 use_lsgan=True,
                 norm='instance',
                 lambda1=10,
                 lambda2=10,
                 learning_rate=2e-4,
                 learning_rate2=2e-6,
                 beta1=0.5,
                 ngf=64
                 ):
        """
        Args:
          batch_size: integer, batch size
          image_size: integer, image size
          lambda1: integer, weight for forward cycle loss (X->Y->X)
          lambda2: integer, weight for backward cycle loss (Y->X->Y)
          use_lsgan: boolean
          norm: 'instance' or 'batch'
          learning_rate: float, initial learning rate for Adam
          beta1: float, momentum term of Adam
          ngf: number of gen filters in first conv layer
        """
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.use_lsgan = use_lsgan
        use_sigmoid = not use_lsgan
        self.batch_size = batch_size
        self.image_size = image_size
        self.learning_rate = learning_rate
        self.learning_rate2 = learning_rate2
        self.beta1 = beta1

        self.is_training = tf.placeholder_with_default(True, shape=[], name='is_training')

        self.G = Generator('G', self.is_training, ngf=ngf, norm=norm, image_size=image_size)
        self.D_Y = Discriminator('D_Y',
                                 self.is_training, norm=norm, use_sigmoid=use_sigmoid)
        self.F = Generator('F', self.is_training, norm=norm, image_size=image_size)
        self.D_X = Discriminator('D_X',
                                 self.is_training, norm=norm, use_sigmoid=use_sigmoid)
        self.C = Classifier("C", self.is_training, norm=norm, use_sigmoid=use_sigmoid)

        self.Uy2x = tf.placeholder(tf.float32, [None, 2], name="Uy2x")
        self.Ux2y = tf.placeholder(tf.float32, [None, 1], name="Ux2y")
        self.x_label = tf.placeholder(tf.float32, [None], name="x_label")
        self.y_label = tf.placeholder(tf.float32, [None], name="y_label")
        self.ClusterX = tf.placeholder(tf.float32, [1, 100], name="ClusterX")
        self.ClusterY = tf.placeholder(tf.float32, [2, 100], name="ClusterY")

        self.x = tf.placeholder(tf.float32,
                                shape=[batch_size, image_size, image_size, 3], name="x")
        self.y = tf.placeholder(tf.float32,
                                shape=[batch_size, image_size, image_size, 3], name="y")
        #self.fake_x=self.F(self.y)
        #self.fake_y=self.G(self.x)
        #self.fake_x = tf.placeholder(tf.float32,
        #                             shape=[batch_size, image_size, image_size, 3], name="fake_x")
        #self.fake_y = tf.placeholder(tf.float32,
        #                             shape=[batch_size, image_size, image_size, 3], name="fake_y")

    def model(self):
        x = self.x
        y = self.y

        #self.fake_x = self.F(self.y)
        #self.fake_y = self.G(self.x)
        cycle_loss = self.cycle_consistency_loss(self.G, self.F, x, y)

        # X -> Y
        fake_y = self.G(x)
        G_gan_loss = self.generator_loss(self.D_Y, fake_y, self.y_label, use_lsgan=self.use_lsgan)
        G_loss = G_gan_loss + cycle_loss
        D_Y_loss = self.discriminator_loss(self.D_Y, y, fake_y, self.y_label, use_lsgan=self.use_lsgan)

        # Y -> X
        fake_x = self.F(y)
        F_gan_loss = self.generator_loss(self.D_X, fake_x, self.x_label, use_lsgan=self.use_lsgan)
        F_loss = F_gan_loss + cycle_loss
        D_X_loss = self.discriminator_loss(self.D_X, x, fake_x, self.x_label, use_lsgan=self.use_lsgan)

        # fuzzy
        Fuzzy_x_loss, feature_x = self.fuzzy_loss(self.C, x, self.Ux2y, self.ClusterX)
        Fuzzy_y_loss,feature_y = self.fuzzy_loss(self.C, fake_x, self.Uy2x, self.ClusterY)
        Disperse_loss = -self.disperse_loss(feature_y, self.Uy2x)
        Fuzzy_loss = Fuzzy_x_loss + Fuzzy_y_loss#+Disperse_loss

        #feature_x = self.C(x)
        #feature_y = self.C(fake_x)
        # summary
        tf.summary.histogram('D_Y/true', self.D_Y(y))
        tf.summary.histogram('D_Y/fake', self.D_Y(self.G(x)))
        tf.summary.histogram('D_X/true', self.D_X(x))
        tf.summary.histogram('D_X/fake', self.D_X(self.F(y)))

        tf.summary.scalar('loss/G', G_gan_loss)
        tf.summary.scalar('loss/D_Y', D_Y_loss)
        tf.summary.scalar('loss/F', F_gan_loss)
        tf.summary.scalar('loss/D_X', D_X_loss)
        tf.summary.scalar('loss/cycle', cycle_loss)
        tf.summary.scalar('loss/Disperse', Disperse_loss)
        tf.summary.scalar('loss/Fuzzy', Fuzzy_loss)

        tf.summary.image('X/generated', utils.batch_convert2int(self.G(x)))
        tf.summary.image('X/reconstruction', utils.batch_convert2int(self.F(self.G(x))))
        tf.summary.image('Y/generated', utils.batch_convert2int(self.F(y)))
        tf.summary.image('Y/reconstruction', utils.batch_convert2int(self.G(self.F(y))))

        return G_loss, D_Y_loss, F_loss, D_X_loss, fake_y, fake_x, Disperse_loss, Fuzzy_loss,feature_x,feature_y

    def optimize(self, G_loss, D_Y_loss, F_loss, D_X_loss, Disperse_loss):
        def make_optimizer(loss, variables, name='Adam'):
            """ Adam optimizer with learning rate 0.0002 for the first 100k steps (~100 epochs)
                and a linearly decaying rate that goes to zero over the next 100k steps
            """
            global_step = tf.Variable(0, trainable=False)
            starter_learning_rate = self.learning_rate
            end_learning_rate = 0.0
            start_decay_step = 100000
            decay_steps = 100000
            beta1 = self.beta1
            learning_rate = (
                tf.where(
                    tf.greater_equal(global_step, start_decay_step),
                    tf.train.polynomial_decay(starter_learning_rate, global_step - start_decay_step,
                                              decay_steps, end_learning_rate,
                                              power=1.0),
                    starter_learning_rate
                )
            )
            tf.summary.scalar('learning_rate/{}'.format(name), learning_rate)

            learning_step = (
                tf.train.AdamOptimizer(learning_rate, beta1=beta1, name=name)
                    .minimize(loss, global_step=global_step, var_list=variables)
            )
            return learning_step

        G_optimizer = make_optimizer(G_loss, self.G.variables, name='Adam_G')
        D_Y_optimizer = make_optimizer(D_Y_loss, self.D_Y.variables, name='Adam_D_Y')
        F_optimizer = make_optimizer(F_loss, self.F.variables, name='Adam_F')
        D_X_optimizer = make_optimizer(D_X_loss, self.D_X.variables, name='Adam_D_X')
        Disperse_optimizer = make_optimizer(Disperse_loss, self.C.variables, name='Adam_D_X')

        #with tf.control_dependencies([G_optimizer, D_Y_optimizer, F_optimizer, D_X_optimizer, Disperse_optimizer]):
        #    return tf.no_op(name='optimizers')
        with tf.control_dependencies([G_optimizer, D_Y_optimizer, F_optimizer, D_X_optimizer]):
            return tf.no_op(name='optimizers')
    def optimize2(self, Fuzzy_loss):
        def make_optimizer2(loss, variables, name='Adam2'):
            """ Adam optimizer with learning rate 0.0002 for the first 100k steps (~100 epochs)
                and a linearly decaying rate that goes to zero over the next 100k steps
            """
            global_step = tf.Variable(0, trainable=False)
            starter_learning_rate = self.learning_rate2
            end_learning_rate = 0.0
            start_decay_step = 100000
            decay_steps = 100000
            beta1 = self.beta1
            learning_rate = (
                tf.where(
                    tf.greater_equal(global_step, start_decay_step),
                    tf.train.polynomial_decay(starter_learning_rate, global_step - start_decay_step,
                                              decay_steps, end_learning_rate,
                                              power=1.0),
                    starter_learning_rate
                )
            )
            tf.summary.scalar('learning_rate2/{}'.format(name), learning_rate)

            learning_step = (
                tf.train.AdamOptimizer(learning_rate, beta1=beta1, name=name)
                    .minimize(loss, global_step=global_step, var_list=variables)
            )
            return learning_step

        Fuzzy_optimizer = make_optimizer2(Fuzzy_loss, self.C.variables, name='Adam_Fuzzy')

        with tf.control_dependencies([Fuzzy_optimizer]):
            return tf.no_op(name='optimizers2')

    def discriminator_loss(self, D, y, fake_y, label, use_lsgan=True):
        """ Note: default: D(y).shape == (batch_size,5,5,1),
                           fake_buffer_size=50, batch_size=1
        Args:
          G: generator object
          D: discriminator object
          y: 4D tensor (batch_size, image_size, image_size, 3)
        Returns:
          loss: scalar
        """
        if use_lsgan:
            # use mean squared error
            error_real = tf.reduce_mean(tf.squared_difference(D(y), label))
            error_fake = tf.reduce_mean(tf.square(D(fake_y)))
        else:
            # use cross entropy
            error_real = -tf.reduce_mean(ops.safe_log(D(y)))
            error_fake = -tf.reduce_mean(ops.safe_log(1 - D(fake_y)))
        loss = (error_real + error_fake) / 2
        return loss

    def generator_loss(self, D, fake_y, label, use_lsgan=True):
        """  fool discriminator into believing that G(x) is real
        """
        if use_lsgan:
            # use mean squared error
            loss = tf.reduce_mean(tf.squared_difference(D(fake_y), label))
        else:
            # heuristic, non-saturating loss
            loss = -tf.reduce_mean(ops.safe_log(D(fake_y))) / 2
        return loss

    def cycle_consistency_loss(self, G, F, x, y):
        """ cycle consistency loss (L1 norm)
        """
        forward_loss = tf.reduce_mean(tf.abs(F(G(x)) - x))
        backward_loss = tf.reduce_mean(tf.abs(G(F(y)) - y))
        loss = self.lambda1 * forward_loss + self.lambda2 * backward_loss
        return loss

    def disperse_loss(self, data, U, DIM=100, m=2):
        data_n = self.batch_size
        cluster_num = 2
        tensor_m = tf.constant(np.ones([data_n, cluster_num], dtype=np.float32) * m)
        UM = tf.pow(U, tensor_m)
        dumpy_sum_num = tf.matmul(UM, data, transpose_a=True)
        dum = tf.expand_dims(tf.reduce_sum(UM, 0), 1)
        g = []
        for i in range(DIM):
            g.append(dum)
        dumpy_sum_dum = tf.concat(g, axis=1)
        clusters = tf.divide(dumpy_sum_num, dumpy_sum_dum)
        c1 = []
        c2 = []
        for i in range(data_n):
            c1.append(tf.expand_dims(clusters[0], 0))
            c2.append(tf.expand_dims(clusters[1], 0))
        cluster_1 = tf.concat(c1, axis=0)
        cluster_2 = tf.concat(c2, axis=0)
        distance = tf.reduce_mean(tf.sqrt(tf.pow(tf.subtract(cluster_1, cluster_2), 2)))
        return distance
    def disperse_loss_original(self, data, C, U, DIM=100, m=2):
        data_n = self.batch_size
        cluster_num = 2
        tensor_m = tf.constant(np.ones([data_n, cluster_num], dtype=np.float32) * m)
        UM = tf.pow(U, tensor_m)
        dumpy_sum_num = tf.matmul(UM, C(data), transpose_a=True)
        dum = tf.expand_dims(tf.reduce_sum(UM, 0), 1)
        g = []
        for i in range(DIM):
            g.append(dum)
        dumpy_sum_dum = tf.concat(g, axis=1)
        clusters = tf.divide(dumpy_sum_num, dumpy_sum_dum)
        c1 = []
        c2 = []
        for i in range(data_n):
            c1.append(tf.expand_dims(clusters[0], 0))
            c2.append(tf.expand_dims(clusters[1], 0))
        cluster_1 = tf.concat(c1, axis=0)
        cluster_2 = tf.concat(c2, axis=0)
        distance = tf.reduce_mean(tf.sqrt(tf.pow(tf.subtract(cluster_1, cluster_2), 2)))
        return distance

    def fuzzy_loss(self, C, x, U, clusters):
        data = C(x)
        data_n = data.get_shape()[0]
        c1 = []
        c2 = []
        for i in range(data_n):
            c1.append(tf.expand_dims(clusters[0], 0))
            if clusters.get_shape()[0] == 2:
                c2.append(tf.expand_dims(clusters[1], 0))
        cluster_1 = tf.concat(c1, axis=0)
        if clusters.get_shape()[0] == 2:
            cluster_2 = tf.concat(c2, axis=0)

        distance_1 = tf.reduce_mean(tf.sqrt(tf.pow(tf.subtract(data, cluster_1), 2)), axis=1)
        if clusters.get_shape()[0] == 2:
            distance_2 = tf.reduce_mean(tf.sqrt(tf.pow(tf.subtract(data, cluster_2), 2)), axis=1)
            distance = tf.concat([tf.expand_dims(distance_1, 1), tf.expand_dims(distance_2, 1)], axis=1)
        else:
            distance = distance_1
        fuzzyLoss = tf.reduce_mean(tf.multiply(distance, U))

        return fuzzyLoss,data

In [57]:
import tensorflow as tf
from datetime import datetime
import os
import logging
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,auc
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.cm as cm
os.environ["CUDA_VISIBLE_DEVICES"] = '1'



Ux = Uy = Cx = Cy = None

x_img_path='/content/drive/MyDrive/Colab Notebook/Data2/banana/train/0_000004.jpg'
x_name=x_img_path.split('/')[-1].split('.')[0]
y_img_path='/home/root123/data/FCGAN/data/'
y_name=y_img_path.split('/')[-1].split('.')[0]

def train():
    if FLAGS.load_model is not None:
        checkpoints_dir = "checkpoints/" + FLAGS.load_model.lstrip("checkpoints/")
        print(checkpoints_dir)
    else:
        logging.info('No model to test, stopped!')
        return

    graph = tf.Graph()
    with graph.as_default():
        cycle_gan = CycleGAN(
            batch_size=FLAGS.batch_size,
            image_size=FLAGS.image_size,
            use_lsgan=FLAGS.use_lsgan,
            norm=FLAGS.norm,
            lambda1=FLAGS.lambda1,
            lambda2=FLAGS.lambda2,
            learning_rate=FLAGS.learning_rate,
            learning_rate2=FLAGS.learning_rate2,
            beta1=FLAGS.beta1,
            ngf=FLAGS.ngf
        )
        G_loss, D_Y_loss, F_loss, D_X_loss, fake_y, fake_x, Disperse_loss, Fuzzy_loss, feature_x, feature_y = cycle_gan.model()

        summary_op = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(checkpoints_dir, graph)
        saver = tf.train.Saver()
    logging.info('Network Built!')

    with tf.Session(graph=graph) as sess:
        checkpoint = tf.train.get_checkpoint_state(checkpoints_dir)
        meta_graph_path = checkpoint.model_checkpoint_path + ".meta"
        restore = tf.train.import_meta_graph(meta_graph_path)
        restore.restore(sess, tf.train.latest_checkpoint(checkpoints_dir))
        step = int(meta_graph_path.split("-")[2].split(".")[0])
        Ux = np.loadtxt(checkpoints_dir + "/Ux" + FLAGS.UC_name + '.txt', delimiter=",")
        Ux = [[x] for x in Ux]
        Uy = np.loadtxt(checkpoints_dir + "/Uy" + FLAGS.UC_name + '.txt', delimiter=",")
        Cx = np.loadtxt(checkpoints_dir + "/Cx" + FLAGS.UC_name + '.txt', delimiter=",")
        Cx = [Cx]
        Cy = np.loadtxt(checkpoints_dir + "/Cy" + FLAGS.UC_name + '.txt', delimiter=",")
        logging.info('Parameter Initialized!')

        #print('Ux',Ux)

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            plots_count=10
            tsne_plot_count=1000
            result_dir='./result'
            fake_dir=os.path.join(result_dir,'fake_xy')
            roc_dir=os.path.join(result_dir,'roc_curves')
            plot_dir=os.path.join(result_dir,'tsne_pca')
            conv_dir=os.path.join(result_dir,'convs')
            #occ_dir=os.path.join(result_dir,'occ_test')
            #utils.prepare_dir(occ_dir)

            x_path = FLAGS.X + FLAGS.UC_name
            x_images, x_id_list, x_len, x_labels, oimg_xs, x_files = get_source_batch(0, 256, 256,
                                                                                      source_dir=x_path)
            y_images, y_id_list, y_len, y_labels, oimg_ys, y_files = get_target_batch(0, 256, 256,
                                                                                      target_dir=FLAGS.Y)
            #Compute Accuracy, tp, tn, fp, fn, f1_score, recall, precision, specificity#
            accuracy, tp, tn, fp, fn, f1_score, recall, precision, specificity=computeAccuracy(Uy,y_labels)
            print("accuracy:%.4f\ttp:%d\ttn:%d\tfp %d\tfn:%d\tf1_score:%.3f\trecall:%.3f\tprecision:%.3f\tspecicity:%.3f\t" %
                  (accuracy, tp, tn, fp, fn,f1_score, recall, precision, specificity))
            #cv2.imshow('201',oimg_ys[201])
            #cv 2.waitKey()
            #draw ROC curves
            '''
            print('y_labels:',np.shape(y_labels))
            print('y_scores:',np.shape(Uy[:,0]))
            fpr,tpr,thresholds=roc_curve(y_labels,Uy[:,1])
            roc_auc=auc(fpr,tpr)
            plt.plot(fpr, tpr)
            plt.xticks(np.arange(0, 1, 0.1))
            plt.yticks(np.arange(0, 1, 0.1))
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            # plt.title("A simple plot")
            plt.show()
            print('fpr:',np.shape(fpr))
            print('tpr:', np.shape(tpr))
            print('thresholds:', np.shape(thresholds))
            '''
            # t-SNE and PCA plots#
            for j in range(plots_count):
                feature_path=os.path.join(checkpoints_dir,'feature_fcgan.npy')
                feature=np.load(feature_path)
                print('feature:',len(feature))
                randIdx = random.sample(range(0, len(y_labels)), tsne_plot_count)
                t_features = []
                t_labels = []
                for i in range(len(randIdx)):
                    t_features.append(feature[randIdx[i]])
                    t_labels.append(y_labels[randIdx[i]])
                # 使用TSNE进行降维处理。从100维降至2维。
                tsne = TSNE(n_components=2, learning_rate=100).fit_transform(t_features)
                #pca = PCA().fit_transform(t_features)
                #设置画布大小
                plt.figure(figsize=(6, 6))
                #plt.subplot(121)
                plt.scatter(tsne[:, 0], tsne[:, 1], c=t_labels)
                #plt.subplot(122)
                #plt.scatter(pca[:, 0], pca[:, 1], c=t_labels)
                plt.colorbar()  # 使用这一句就可以分辨出，颜色对应的类了！神奇啊。
                utils.prepare_dir(plot_dir)
                plt.savefig(os.path.join(plot_dir,'plot{}.pdf'.format(j)))


            for i in range(10):
            #if True:
            #Cross Domain Image Generation#

                x_img,_,x_oimg = get_single_img(x_img_path)
                y_path=os.path.join(y_img_path,str(i+1)+'.jpg')
                y_img,_,y_oimg=get_single_img(y_path)
                fake_y_eval,fake_x_eval, conv_y_eval = sess.run(
                [fake_y, fake_x,  tf.get_collection('conv_output')],
                feed_dict={cycle_gan.x: x_img, cycle_gan.y: y_img})
                #print(np.shape(fake_y_eval))
                #print(np.shape(fake_x_eval))
                #print(np.shape(conv_y_eval))
                plot_fake_xy(fake_y_eval[0], fake_x_eval[0], x_name, str(i+1), x_oimg,y_oimg,fake_dir)
                print('processing:',i)

            #Feature Map Visualization#

                print('conv_len:', len(conv_y_eval))
                print('conv_shape:',np.shape(conv_y_eval[0]))
                id_y_dir=os.path.join(conv_dir, str(y_name))
                #utils.prepare_dir()
                for i, c in enumerate(conv_y_eval):
                    #conv_i_dir=os.path.join(id_y_dir,'_layer_'+str(i))
                    plot_conv_output(c,i,id_y_dir)
                #print(os.path.join(id_y_dir, 'y.png'))
                cv2.imwrite(os.path.join(id_y_dir, 'y.png'), y_oimg)

            #Occlusion Test#
            '''
            if True:
                t_imgs, t_lbs, t_img = get_single_img(t_img_path)
                #s_imgs, s_lbs, t_img = get_single_img(s_img_path)
                width=np.shape(t_imgs[0])[0]
                height=np.shape(t_imgs[0])[1]
                #print('width:',width)
                #print('height:', height)
                data=generate_occluded_imageset(t_imgs[0],width=width,height=height,occluded_size=16)
                #print(data.shape[0])
                #print('Cy:',Cy)
                u_ys=np.empty([data.shape[0]],dtype='float64')
                occ_map=np.empty((width,height),dtype='float64')
                print(occ_map.shape)
                cnt=0
                feature_y_eval = sess.run(
                    feature_y,
                    feed_dict={cycle_gan.y: [data[0]]})
                # print(feature_y_eval)
                idx_u = 0
                u_y0 = cal_U(feature_y_eval[0], Cy, 2, 2)[idx_u]
                occ_value=0
                print('u_y0:',u_y0)
                for i in range(width):
                    print('collum idx:',i)
                    print(str(cnt) + ':' + str(occ_value))
                    for j in range(height):
                        feature_y_eval = sess.run(
                            feature_y,
                            feed_dict={cycle_gan.y: [data[cnt+1]]})
                        # print(feature_y_eval)
                        u_y = cal_U(feature_y_eval[0], Cy, 2, 2)[idx_u]
                        #print('u_y0:', u_y0)
                        #print('u_y:',u_y)
                        occ_value=u_y0-u_y
                        occ_map[i,j]=occ_value
                        #print(str(cnt)+':'+str(occ_value))
                        cnt+=1
                occ_map_path=os.path.join(occ_dir,'occlusion_map.txt')
                np.savetxt(occ_map_path, occ_map, fmt='%0.20f')
                cv2.imwrite(os.path.join(occ_dir, 'occ_test.png'), oimg_ys[id_y])
                draw_heatmap(occ_map_path=occ_map_path,ori_img=t_img,save_dir=os.path.join(occ_dir,'heatmap.png'))
            '''


        except KeyboardInterrupt:
            logging.info('Interrupted')
            coord.request_stop()
        except Exception as e:
            coord.request_stop(e)
        finally:
            #save_path = saver.save(sess, checkpoints_dir + "/model.ckpt", global_step=step)
            #np.savetxt(checkpoints_dir + "/Uy" + FLAGS.UC_name + '.txt', Uy, fmt="%.20f", delimiter=",")
            #np.savetxt(checkpoints_dir + "/Cy" + FLAGS.UC_name + '.txt', Cy, fmt="%.20f", delimiter=",")
            #np.savetxt(checkpoints_dir + "/Ux" + FLAGS.UC_name + '.txt', Ux, fmt="%.20f", delimiter=",")
            #np.savetxt(checkpoints_dir + "/Cx" + FLAGS.UC_name + '.txt', Cx, fmt="%.20f", delimiter=",")
            logging.info("stopped")
            # When done, ask the threads to stop.
            coord.request_stop()
            coord.join(threads)


def main(unused_argv):
    train()


In [61]:
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, Flatten, Dense, MaxPool2D

model = Sequential()
model.add(MaxPool2D((2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(), metrics=['acc'])

mc = ModelCheckpoint('en_iyi.h5',save_best_only=True,monitor='val_loss',mode='min')

